# Self study 2


In this self-study we build an index that supports Boolean search over the web pages that you crawl with the crawler from the 1st self study. You can continue to just extract the titles of the web-pages you crawl, or you can be more adventurous and look at the whole text that you get from the .get_text() method of a BeautifulSoup parser. In either case, the collection of texts from the crawled web-pages is you corpus. You should then:

- construct the vocabulary of terms for your corpus
- build an 'inverted' index for your vocabulary
- implement Boolean search for your index (perhaps only for a limited set of Boolean queries)

In [ ]:
# Some things already used in self study 1:
import requests
from bs4 import BeautifulSoup


A useful resource is the nltk natural language processing package:
https://www.nltk.org/
which provides methods for tokenization, stemming, and much more (the 'punkt' package is needed for tokenization):

In [ ]:
import nltk
nltk.download('punkt')

Now let's use the title string of the AAU homepage as an example:

In [ ]:
r=requests.get('https://www.aau.dk/')
r_parse = BeautifulSoup(r.text, 'html.parser')
string=r_parse.find('title').string
print(string)

We can tokenize:

In [ ]:
tokens=nltk.word_tokenize(string)
for t in tokens:
    print(t)

And we can stem:

In [ ]:
ps=nltk.PorterStemmer()
for t in tokens:
    print(ps.stem(t))



For Danish language the Porter stemmer will not be terribly useful! There is also a Danish option:

In [ ]:
from nltk.stem.snowball import SnowballStemmer

dstemmer=SnowballStemmer("danish")

In [ ]:
for t in tokens:
    print(dstemmer.stem(t))


What is most useful for you depends on which websites you crawl. It is not essential for the exercise that the stemming always is the best possible ...!